# Prose corpus segmentation

In [1]:
from nltk import word_tokenize
import numpy as np
import pandas as pd
from typing import Dict, List, Optional, Tuple, Union
from utils import build_chunks, split_texts_into_segments, unify_texts_amount

In [2]:
corpus = pd.read_csv("../data/corpora/prose_corpus.csv")

### hyperparameter

Segments: 10000, 25000

In [3]:
segments = 10000

### Splitting into segments

In [4]:
%%time
new_corpus = split_texts_into_segments(corpus,
                                       max_segments=3,
                                       n=segments,
                                       same_len=True)
new_corpus.head()

CPU times: user 4min 6s, sys: 405 ms, total: 4min 7s
Wall time: 4min 7s


,filename,author,title,year,textlength,text
0,Jakob_Christoph_Heer_-_Der_Wetterwart_(1905)_1,Jakob Christoph Heer,Der Wetterwart_1,1905,10000,1925 I Die feierliche Abendhelle steht über de...
1,Jakob_Christoph_Heer_-_Der_Wetterwart_(1905)_2,Jakob Christoph Heer,Der Wetterwart_2,1905,10000,"spüre es , die Ingenieurkunst wäre mein innigs..."
2,Jakob_Christoph_Heer_-_Der_Wetterwart_(1905)_3,Jakob Christoph Heer,Der Wetterwart_3,1905,10000,uns stets ein treuer Nachbar gewesen . « Ich h...
3,Johannes_Richard_zur_Megede_-_Der_Ueberkater_-...,Johannes Richard zur Megede,Der Ueberkater - Band 1_1,1904,10000,Erster Band Fräulein Elisabeth von Skal in dan...
4,Johannes_Richard_zur_Megede_-_Der_Ueberkater_-...,Johannes Richard zur Megede,Der Ueberkater - Band 1_2,1904,10000,. Sollte dieses Dorschgesicht am Ende einem fe...


In [38]:
new_corpus.shape

(1464, 6)

In [63]:
shorten_new_corpus = new_corpus.groupby('author').filter(lambda x : len(x)>20)

In [64]:
shorten_new_corpus.shape

(696, 6)

In [67]:
corpus.shape

(488, 6)

In [66]:
#shorten_new_corpus.to_csv(f"../data/corpora/prose_corpus_{segments}seg.csv", index=False)